# Efficient Compressed Sensing SENSE pMRI Reconstruction With Joint Sparsity Promotion
Il Yong Chun, Ben Adcock, and Thomas M. Talavage, 2015

## Julia Implementation

In [1]:
using LinearAlgebra, MAT, FFTW, Wavelets, Random, Plots

┌ Info: Precompiling Wavelets [29a6e085-ba6d-5f35-a997-948ac2efa89a]
└ @ Base loading.jl:1260
┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1260


### Utility functions 

In [2]:
function SoS(x)
    return sqrt.(sum(x.^2, dims=3)[:,:])
end

SoS (generic function with 1 method)

In [3]:
function Jsoftshrink(x, lambda)
   return (x./norm(x)) .* min(norm(x) - lambda, 0)
end

Jsoftshrink (generic function with 1 method)

### Initialisation

In [4]:
# Load data 
file = matopen("data/brain_coil.mat");
coilScans = read(file, "brain_coil_tmp");
close(file);
file = matopen("data/coil_sensitivity_map.mat");
coilSensitivityMaps = read(file, "coil_map");
close(file);

# K-spaces
height, width, C = size(coilScans);
y_original = reshape(fftshift(fft(coilScans)), :, 1); # Observed K-space, vectorised and all coils concatenated
y = y_original;
N = height*width; # Length of vectorised images

# Diagonalised Sensitivity Maps
vectorisedMaps = reshape(coilSensitivityMaps, :, C);
S = Diagonal(vectorisedMaps[:,1]);
for coil=2:C
    S = [S; Diagonal(vectorisedMaps[:,2])];
end

In [5]:
# Define hyperparameters 
alpha = 0.05;
beta = 2*alpha;
gamma = 2*alpha;
v = 2*alpha;
nInner = 1; # Number of inner Bregman iterations 
nOuter = 10; # Number of outer Bregman iterations 
# delta = # Alternatively can define threshold for stopping condition instead of nOuter

In [28]:
# Define important matrices
I_c = Diagonal(ones(C));
I_n = Diagonal(ones(N));
file = matopen("data/gradients.mat");
G1 = read(file, "GH"); # Horizontal gradient operator
G2 = read(file, "GV"); # Vertical gradient operator 
close(file);
G1c = kron(I_c, G1); # Kroneckered G1 for vector operations
G2c = kron(I_c, G2); # Kroneckered G2 for vector operations

UndefVarError: UndefVarError: kron! not defined

In [41]:
size(G1c)

(79300, 76800)

In [31]:
Phi = fft(I_n, 1); # Fourier transform
#Phi_c = kron(I_c, Phi);
Psi =  dct(I_n, 1); # Sparsifying transform
@show size(Psi)
#Psi_c = kron(I_c, Psi); # Kroneckered Psi for vector operations

size(Psi) = (15360, 15360)


In [8]:
m = convert(UInt16, 0.6*N);
p = [ones(m); zeros(N-m)];
shuffle(p);
P_Omega = Diagonal(p); # Diagonal projection matrix
P_Omega_c = kron(I_c, P_Omega); # Kroneckered P_Omega for vector operations

In [9]:
F_Omega_c = P_Omega_c * kron(I_c, Phi);

OutOfMemoryError: OutOfMemoryError()

In [10]:
# Precompute values
Lambda = alpha*P_Omega_c'*P_Omega_c + (beta + v)*I + gamma*Phi_c*(G1c'*G1c + G2c'*G2c)*Phi_c';

# Initialise values 
x = SoS(Phi_c'*y);
ds = zeros(Float64, (N*C, 1));
dpsi = zeros(Float64, (N*C, 1));
d1 = zeros(Float64, (N*C, 1));
d2 = zeros(Float64, (N*C, 1));
bs = zeros(Float64, (N*C, 1));
bpsi = zeros(Float64, (N*C, 1));
b1 = zeros(Float64, (N*C, 1));
b2 = zeros(Float64, (N*C, 1));

UndefVarError: UndefVarError: Phi_c not defined

### Recursive Computation

In [11]:
for nOut = 1:nOuter
    for nIn = 1:nInner
        x = pinv(S'*S)*S'*(ds - bs);
        ds = Phi_c'*pinv(Lambda)*Phi_c*(alpha*F_Omega'*y + beta*Psi_c'*(dpsi-bpsi) + gamma*G1c'*(d1-b1) + gamma*G2c'*(d1-b2) + v*(S*x+bs));
        dpsi = Jsoftshrink(Psi_c*ds + bpsi, 1/beta);
        d1 = Jsoftshrink(G1c*ds+b1, 1/gamma);
        d2 = Jsoftshrink(G2c*ds+b2, 1/gamma);
        bs = bs + S*x - ds;
        bpsi = bpsi + Psi_c*ds - dpsi;
        b1 = b1 + G1c*ds - d1;
        b2 = b2 + G2c*ds - d2;
    end
    y = y + y_original - F_Omega_c*S*x;
end

UndefVarError: UndefVarError: ds not defined

### Visualisation